In [141]:
from src.my_sort import p_distributions, thresholds, plot_strategy_domains, plot_partition_tree, mP
from sympy.stats import Binomial, density
from sympy import Symbol, integrate, solve, Interval, Rational
from math import comb
import numpy as np
from matplotlib import pyplot as plt
from sympy import lambdify, Symbol, Interval

sys.set_int_max_str_digits(0)
n1 = Symbol("n1", domain=Interval(0, 1))

In [110]:
class DiscreteSort(metaclass=LoggingMeta):

    def __init__(self, rational=False):
        self.rational = rational

    @staticmethod
    def hypergeometric(n, k, maxi): 
        # population, population considerada valida , turnos left, aciertos requeridos
        return density(Hypergeometric('H', maxi, maxi-n1, n))(k)

    @lru_cache # cache with self = bad idea death
    def mP(self, n, maxi):
        if n < 2 or n >= maxi:
            return 1
        B, I = self.thresholds(n, maxi) 
        return simplify(sum([Sum(B[i], (n1, I[i]+1,I[i+1])) for i in range(n)])/maxi)

    @lru_cache
    def p_distributions(self, n, maxi):
        # n is the number of boxes
        result = [self.hypergeometric(n-1, i, maxi) * self.mP(n-1-i, maxi-n1) * self.mP(i, n1)-1 for i in range(n)]
        if self.rational: # not sure if needed anymore
            return result
        return [simplify(r).evalf() for r in result]

    @lru_cache
    def thresholds(self, n, maxi):
        B = self.p_distributions(n, maxi)
        I = [0]
        for i in range(0, n-1):
            #  should be reduce_inequalities with constrains but do not work well
            sol = solve(B[i+1]-B[i], n1, rational=self.rational)
            sol = sol[1] if sol[0] == 0 else sol[0]
            sol = int(sol)
            I.append(sol)
        I.append(maxi)
       
        return B, I

In [111]:
sorti = DiscreteSort(rational=False)


Calling method __init__ with args () and kwargs {'rational': False}


In [87]:
# caso 2
# {1}{0}
# {0}{1}

In [26]:
def hypergeometric(n, k, maxi): 
    return density(Hypergeometric('H', maxi, k, maxi-n1))(n1)

def k_dist(n, k, maxi):
    assert k <= n, 'k must be less or equal than n'
    return density(Hypergeometric('H', maxi, k, n))(n1)

In [34]:
k_dist(1,0, 1000)

binomial(0, n1)*binomial(1000, 1 - n1)/1000

In [35]:
k_dist(1,1, 1000)

binomial(1, n1)*binomial(999, 1 - n1)/1000

In [32]:
b=2 # buckets
max_n = 4

P1L = hypergeometric(b-1, 1, max_n) 
P1R = hypergeometric(b-1, 0, max_n)
P1 = (Sum(P1L, (n1, 1, max_n//2)) + Sum(P1R, (n1, max_n//2+1, max_n)))/max_n
P1.evalf()

0.187500000000000

In [18]:
P1

Sum(binomial(0, n1)*binomial(4, 1 - n1)/4, (n1, 1, 2))/4 + Sum(binomial(1, n1)*binomial(3, 1 - n1)/4, (n1, 3, 4))/4

In [144]:
density(Hypergeometric('H', 10, 7, 2)).dict

{0: 1/15, 1: 7/15, 2: 7/15}

In [142]:
float(density(Hypergeometric('H', 10, 7, 2)).dict[2])

0.4666666666666667

In [143]:
float(density(Hypergeometric('H', 10, 7, 2)).dict[1])

0.4666666666666667